In [101]:
import os
import pandas as pd
from IPython.display import display
from datetime import timedelta
from datetime import date as convert_to_date
from sklearn.preprocessing import normalize
import numpy as np


ROOT_DIR= os.getcwd()
data_dir = ROOT_DIR + "/StudentLife Data"
student_list = os.listdir(data_dir)

student_list = [ _ for _ in student_list if "student" in _]

def adjust_stress_values(stress_level):
    mapping = {
        1: 2,
        2: 3,
        3: 4,
        4: 1,
        5: 0
    }

    try:
        return mapping[stress_level]
    except KeyError:
        return None

In [102]:
for student in student_list:
    print(student)

    files = os.listdir(data_dir + "/{}".format(student))
    files = [ _ for _ in files if "train_x.csv" in _]
    
    for file in files:
        

        
        csv_file_name = "{}/{}/{}".format(data_dir, student, file)
        
        feature_train_x = pd.read_csv("{}/{}/{}".format(data_dir, student, file),
                                      skip_blank_lines=False, 
                                      parse_dates= True,
                                      index_col=1
                                     )
        feature_train_x = feature_train_x.iloc[:,1:]
        resampled_feature_train_x = feature_train_x.resample('2T').max()
        resampled_feature_train_x.iloc[:, :-1] = resampled_feature_train_x.iloc[:, :-1].fillna(method="ffill")
        
        
        # Parse Min and Max Date, Convert them to string.
        start_date = resampled_feature_train_x.index.min()
        end_date = resampled_feature_train_x.index.max()
        start_date = start_date.to_datetime()
        end_date = end_date.to_datetime()
        
        start_date = convert_to_date(start_date.year, start_date.month, start_date.day)
        end_date = convert_to_date(end_date.year, end_date.month, end_date.day) + timedelta(days=1)
        
        ix = pd.date_range(start=start_date, end=end_date, freq='2T')
        resampled_feature_train_x = resampled_feature_train_x.reindex(ix).iloc[:-1,1:]
        
        # Filling NA Values.
        resampled_feature_train_x.iloc[:, :-1] = resampled_feature_train_x.iloc[:, :-1].fillna(method='ffill')
        resampled_feature_train_x.iloc[:, :-1] = resampled_feature_train_x.iloc[:, :-1].fillna(method='bfill')
        
        unique_dates = list(resampled_feature_train_x.index.map(lambda t: t.date()).unique())
        
        x = []
        mask = []
        y = []
        
        for idx, date in enumerate(unique_dates):
            
            days_train_x = resampled_feature_train_x.loc[str(date) : str(date)].iloc[:, :-1]
            days_train_y = resampled_feature_train_x.loc[str(date) : str(date)].iloc[:, -1]
            days_train_y = days_train_y.apply(adjust_stress_values)
            
            days_train_y.reset_index(drop=True, inplace=True)
            days_train_y_index_mask = days_train_y.notnull()
            days_train_y = days_train_y[days_train_y_index_mask]
            
            days_train_x = days_train_x.as_matrix()
            days_train_y_index_mask = days_train_y_index_mask.as_matrix()
#             print("Mask Shape:", days_train_y_index_mask.shape)
            
            #Normalize Days Training Data
            days_train_x = normalize(days_train_x)
            
            x.append(days_train_x)
            mask.append(days_train_y_index_mask)
            
            y = y + list(days_train_y)
        
        # Stacking All the days worth data.
        train_x = np.stack(x, axis=0)
        train_mask = np.stack(mask, axis=0)
        train_mask = train_mask.astype(int)
        train_y = np.array(y)
        
#         print("TrainX shape {}, Mask Shape {}".format(train_x.shape, train_mask.shape))
        
        np.savez("{}/{}/{}".format(data_dir, student, file[:-4]), input_seq=train_x, mask=train_mask, target=train_y )
        

student 31


/home/abhinav/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2910: FutureWarning: to_datetime is deprecated. Use self.to_pydatetime()
  exec(code_obj, self.user_global_ns, self.user_ns)


student 0
student 1
student 10
student 12
student 14
student 15
student 16
student 17
student 18
student 19
student 2
student 20
student 22
student 23
student 24
student 25
student 27
student 3
student 30
student 32
student 33
student 34
student 35
student 36
student 39
student 4
student 41
student 42
student 43
student 44
student 45
student 46
student 47
student 49
student 5
student 50
student 51
student 52
student 53
student 54
student 56
student 57
student 58
student 59
student 7
student 8
student 9


In [96]:
import torch
a = torch.randn( 2, 3, 5)
mask = torch.ByteTensor([[1, 0, 0], [0,1,0]])




mask = torch.unsqueeze(mask, dim=2)
mask = mask.expand(2,3,5)
print(torch.masked_select(a, mask))




-1.5198
 0.7118
-0.9143
-0.8680
-1.6288
 0.9820
-0.4699
 0.7407
 1.2879
 1.8683
[torch.FloatTensor of size 10]

